# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [206]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [207]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print (df.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [208]:
# frequency table for prestige and whether or not someone was admitted
df1=pd.crosstab(index=df['prestige'], columns=df['admit'])
print(df1)

admit      0   1
prestige        
1.0       28  33
2.0       95  53
3.0       93  28
4.0       55  12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [209]:
dummy_ranks = pd.get_dummies(df["prestige"],prefix ='prestige')
df = pd.concat([df, dummy_ranks], axis=1)
df.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,3.0,0,0,1,0
1,1,660.0,3.67,3.0,0,0,1,0
2,1,800.0,4.00,1.0,1,0,0,0
3,1,640.0,3.19,4.0,0,0,0,1
4,0,520.0,2.93,4.0,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: n-1=4-1= 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [210]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_1.0':])
print (handCalc.head())

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


In [211]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
df3=pd.crosstab(index=df['prestige_1.0'], columns=df['admit'])
print(df3)

admit           0   1
prestige_1.0         
0             243  93
1              28  33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [212]:
odds1=33/28
print (odds1)

1.1785714285714286


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [213]:
oddsother=(93)/(243)
print(oddsother)

0.38271604938271603


#### 3.3 Calculate the odds ratio

In [214]:
odds1/oddsother

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: An applicant from an undergraduate school with a top ranking is over 3.08 times more likely to be admitted than an applicant from a non-top ranked undergraduate school

#### 3.5 Print the cross tab for prestige_4

In [215]:
df2=pd.crosstab(index=df['prestige_4.0'], columns=df['admit'])
print(df2)

admit           0    1
prestige_4.0          
0             216  114
1              55   12


#### 3.6 Calculate the OR 

In [216]:
odds_ratio_4= (12/55)/(114/216)
print (odds_ratio_4)

0.4133971291866028


#### 3.7 Write this finding in a sentence

Answer: The odds of admission of applicants from undergraduate schools with a 4 ranking was .41 of applicants from undergraduate schools with higher rankings.

## Part 4. Analysis

In [217]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_2.0':])
print (data.head())

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [218]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [219]:
train_cols = data.columns[1:].values
print (train_cols)

['gre' 'gpa' 'prestige_2.0' 'prestige_3.0' 'prestige_4.0' 'intercept']


#### 4.2 Fit the model

In [220]:
import statsmodels.api as smf
X = data[train_cols]
y = data['admit']
lm = smf.Logit(y , X).fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [221]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 06 Dec 2017   Pseudo R-squ.:                 0.08166
Time:                        18:54:55   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [222]:
print (lm.params)

gre             0.002218
gpa             0.779337
prestige_2.0   -0.680137
prestige_3.0   -1.338677
prestige_4.0   -1.553411
intercept      -3.876854
dtype: float64


In [225]:
params = lm.params
conf = lm.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print (np.exp(conf))

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


In [197]:
np.exp(lm.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

#### 4.5 Interpret the OR of Prestige_2

Answer: The odds of acceptance from a #2 Prestige school is .5 to 1 compared to a #1 Prestige school

#### 4.6 Interpret the OR of GPA

Answer: For every unit increase in GPA, an applicant's odds of acceptance increase by 2.18

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [170]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype
    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)
    m = int(n / arrays[0].size)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [188]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print (gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])
# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [181]:
combos.head()

,0,1,2
0,220.0,2.260000,1.0
1,220.0,2.260000,2.0
2,220.0,2.260000,3.0
3,220.0,2.260000,4.0
4,220.0,2.453333,1.0


#### 5.1 Recreate the dummy variables

In [189]:
# recreate the dummy variables
combos.columns=['gre', 'gpa', 'prestige']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix ='prestige')
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa']
combos = combos[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_2.0':])

In [190]:
combos.head()

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,0,0,0
1,220.0,2.260000,1,0,0
2,220.0,2.260000,0,1,0
3,220.0,2.260000,0,0,1
4,220.0,2.453333,0,0,0


In [191]:
combos['intercept']=1

#### 5.2 Make predictions on the enumerated dataset

In [204]:
combos['admit_pred']=lm.predict(combos[train_cols])

In [205]:
print (combos.tail())

       gre       gpa  prestige_2.0  prestige_3.0  prestige_4.0  intercept  \
395  800.0  3.806667             0             0             1          1   
396  800.0  4.000000             0             0             0          1   
397  800.0  4.000000             1             0             0          1   
398  800.0  4.000000             0             1             0          1   
399  800.0  4.000000             0             0             1          1   

     admit_pred  
395    0.334286  
396    0.734040  
397    0.582995  
398    0.419833  
399    0.368608  


#### 5.3 Interpret findings for the last 4 observations

Answer: With a 4.0 GPA and 800 GRE, an applicant from a Prestige 1 school has an almost 75% chance of being admitted.  A Prestige 2 school applicant has a 58% probability of admission, a Prestige 3 a 42% chance, and a Prestige 4 only a 36% probability.  The Prestige 1 applicant is twice as likely to be admitted as a Prestige 4 applicant with the same GPA and GRE. 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.